In [1]:
import keras
import tensorflow

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import tensorflow

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='IN74xk-i8nv6ctX_ZjA64ugCiNpjQFkrlI_Asfjun7Rx',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'ibmcloudmodeltraining-donotdelete-pr-huqvvutovpoiwj'
object_key = 'archive (2).zip'

streaming_body_4 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_4.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   rotation_range=180,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
pwd

'/home/wsuser/work'

In [7]:
cd /home/wsuser/work/archive

/home/wsuser/work/archive


In [8]:
ls archive/Dataset/Dataset

ls: cannot access 'archive/Dataset/Dataset': No such file or directory


In [9]:
x_train = train_datagen.flow_from_directory(r'/home/wsuser/work/archive/Dataset/Dataset/test_set',
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='binary')

Found 320 images belonging to 2 classes.


In [10]:
x_test = train_datagen.flow_from_directory(r'/home/wsuser/work/archive/Dataset/Dataset/train_set',
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='binary')

Found 436 images belonging to 2 classes.


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten

In [12]:
model = Sequential()
model.add(Convolution2D(32, (3,3), input_shape=(128, 128, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(150,activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [13]:
model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [14]:
model.fit(x_train, steps_per_epoch=10, epochs=10, validation_data=x_test, validation_steps=4)

Epoch 1/10
10/10 [==============================] - 20s 2s/step - loss: 1.3456 - accuracy: 0.7250 - val_loss: 0.4675 - val_accuracy: 0.7656
Epoch 2/10
10/10 [==============================] - 17s 2s/step - loss: 0.2788 - accuracy: 0.8781 - val_loss: 0.2582 - val_accuracy: 0.8984
Epoch 3/10
10/10 [==============================] - 19s 2s/step - loss: 0.2061 - accuracy: 0.9187 - val_loss: 0.2120 - val_accuracy: 0.8906
Epoch 4/10
10/10 [==============================] - 19s 2s/step - loss: 0.2211 - accuracy: 0.9062 - val_loss: 0.2123 - val_accuracy: 0.8906
Epoch 5/10
10/10 [==============================] - 20s 2s/step - loss: 0.1682 - accuracy: 0.9281 - val_loss: 0.2076 - val_accuracy: 0.9297
Epoch 6/10
10/10 [==============================] - 19s 2s/step - loss: 0.1689 - accuracy: 0.9281 - val_loss: 0.2751 - val_accuracy: 0.8516
Epoch 7/10
10/10 [==============================] - 18s 2s/step - loss: 0.1643 - accuracy: 0.9344 - val_loss: 0.2174 - val_accuracy: 0.8906
Epoch 8/10
10/10 [==

In [15]:
model.save("model.h5")

In [16]:
!tar -zcvf model.tgz model.h5

model.h5


In [17]:
!pip install watson-machine-learning-client

In [18]:
from ibm_watson_machine_learning import APIClient

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "8yTdmLLPAug9Gb_hPmHhtHH6g8166qNDpB0pJbvP8VhO"
}

client = APIClient(credentials)

In [19]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [20]:
 space = client.spaces.get_details()
 space

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/9ec34b387be043edadd71c2a9ca35626:2b62c55d-6d9a-4eb3-89f8-14643df03296::',
      'guid': '2b62c55d-6d9a-4eb3-89f8-14643df03296',
      'name': 'Watson Machine Learning-uo',
      'type': 'machine_learning'}],
    'description': '',
    'name': 'IBM Cloud Mobel Training',
    'scope': {'bss_account_id': '9ec34b387be043edadd71c2a9ca35626'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': '04fb032d-6e22-4f48-973f-38d650b24ca6',
      'bucket_region': 'us-south',
      'credentials': {'admin': {'access_key_id': '12a1cf9f9d644ef7863bae49ec220ee5',
        'api_key': 'dobeDa-dNBtmPqyyBsjItqvk-l9ch-CF7mP0UHNKXPLD',
        'secret_access_key': 'd8df138746d5d0a2b0791dafb3bac2cf48c996187fd3098f',
        'service_id': 'ServiceId-79cae6c7-5169-4940-a3ab-2f6b4711a2fa'},
       'editor': {'access_key_id': 'a1b5620ddd7c4031bcce950142617e20',
    

In [21]:
space_uid = guid_from_space_name(client, 'IBM Cloud Mobel Training')
print("Space UID: ", space_uid)

Space UID:  eb133025-bf61-4f1d-ade1-97cb578b4323


In [22]:
client.set.default_space(space_uid)

'SUCCESS'

In [23]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [24]:
model_details = client.repository.store_model(model="model.tgz", meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

In [25]:
client.repository.download(model_id, "model.tar.gz")

Successfully saved model content to file: 'model.tar.gz'


'/home/wsuser/work/archive/model.tar.gz'